In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Training.csv')

In [3]:
df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,Unnamed: 133
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo,NaN
4916,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,Acne,NaN
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Urinary tract infection,NaN
4918,0,1,0,0,0,0,1,0,0,0,...,0,1,1,1,1,0,0,0,Psoriasis,NaN


In [4]:
x = df.iloc[:, 0:132].values
y = df.iloc[:, 132].values

In [5]:
x

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1]], dtype=int64)

In [6]:
y

array(['Fungal infection', 'Fungal infection', 'Fungal infection', ...,
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [7]:
def one_hot_encode(data):
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder()
    return enc.fit_transform(data.reshape(-1,1)).toarray()

In [8]:
y = one_hot_encode(y)

In [9]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
print(x.shape)
print(y.shape)

(4920, 132)
(4920, 41)


In [11]:
input_dim = x.shape[1]
output_dim = y.shape[1]
hidden_layer_dim = 50
hidden_layers = 2

learning_rate = 0.001
lambda_reg = 0.01
epochs = 1000

samples = x.shape[0]

print("Input dimensions: ", input_dim)
print("Output dimensions: ", output_dim)
print("Number of samples: ", samples, "\n")

print("w1 dimensions: ", (input_dim, hidden_layer_dim))
print("w2 dimensions: ", (hidden_layer_dim, hidden_layer_dim))
print("w3 dimensions: ", (hidden_layer_dim, output_dim), "\n")

print("z1 and a1 dimensions: ", (hidden_layer_dim, samples))
print("z2 and a2 dimensions: ", (hidden_layer_dim, samples))
print("z3 and a3 dimensions: ", (output_dim, samples))

Input dimensions:  132
Output dimensions:  41
Number of samples:  4920 

w1 dimensions:  (132, 50)
w2 dimensions:  (50, 50)
w3 dimensions:  (50, 41) 

z1 and a1 dimensions:  (50, 4920)
z2 and a2 dimensions:  (50, 4920)
z3 and a3 dimensions:  (41, 4920)


In [12]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return x*(1-x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [13]:
'''
    Weight initialization using Xavier initialization
'''
def init_network():
    w3 = np.random.randn(hidden_layer_dim, output_dim) * np.sqrt(1/(hidden_layer_dim + output_dim))
    w2 = np.random.randn(hidden_layer_dim, hidden_layer_dim) * np.sqrt(1/(hidden_layer_dim + hidden_layer_dim))
    w1 = np.random.randn(input_dim, hidden_layer_dim) * np.sqrt(1/(input_dim + hidden_layer_dim))

    b1 = np.zeros((1, hidden_layer_dim))
    b2 = np.zeros((1, hidden_layer_dim))
    b3 = np.zeros((1, output_dim))

    return {'w1': w1, 'w2': w2, 'w3': w3, 'b1': b1, 'b2': b2, 'b3': b3}

In [14]:
def retrieve(model_dict):
    w1 = model_dict['w1']
    w2 = model_dict['w2']
    w3 = model_dict['w3']
    b1 = model_dict['b1']
    b2 = model_dict['b2']
    b3 = model_dict['b3']

    return w1, w2, w3, b1, b2, b3

In [15]:
def forward(x, model_dict):
    w1, w2, w3, b1, b2, b3 = retrieve(model_dict)

    z1 = x.dot(w1) + b1
    a1 = sigmoid(z1)

    z2 = a1.dot(w2) + b2
    a2 = sigmoid(z2)

    z3 = a2.dot(w3) + b3
    y_pred = softmax(z3)

    return y_pred, a1, a2

In [16]:
def predict(x, model_dict):
    y_pred, _, _ = forward(x, model_dict)
    return y_pred

In [17]:
def loss(y_pred, y, model_dict):
    w1, w2, w3, _, _, _ = retrieve(model_dict)

    m = y.shape[0]
    cross_entropy_loss = -np.sum(y * np.log(y_pred)) / m
    regularization_loss = (lambda_reg/(2*m)) * (np.sum(np.square(w1)) + np.sum(np.square(w2)) + np.sum(np.square(w3)))

    return cross_entropy_loss + regularization_loss

In [18]:
def backpropagation(x, y, model_dict, epochs):
    for i in range(epochs):
        w1, w2, w3, b1, b2, b3 = retrieve(model_dict)
        y_pred, a1, a2 = forward(x, model_dict)

        delta4 = y_pred - y 
        dw3 = np.dot(a2.T, delta4)
        db3 = np.sum(delta4, axis=0, keepdims=True)

        delta3 = np.dot(delta4, w3.T) * sigmoid_derivative(a2)
        dw2 = np.dot(a1.T, delta3)
        db2 = np.sum(delta3, axis=0)

        delta2 = np.dot(delta3, w2.T) * sigmoid_derivative(a1)
        dw1 = np.dot(x.T, delta2)
        db1 = np.sum(delta2, axis=0)

        dw3 += lambda_reg * np.sum(w3)
        dw2 += lambda_reg * np.sum(w2)
        dw1 += lambda_reg * np.sum(w1)

        w1 -= learning_rate * dw1
        w2 -= learning_rate * dw2
        w3 -= learning_rate * dw3

        b1 -= learning_rate * db1
        b2 -= learning_rate * db2
        b3 -= learning_rate * db3

        model_dict = {'w1': w1, 'w2': w2, 'w3': w3, 'b1': b1, 'b2': b2, 'b3': b3}

        if i % 50 == 0:
            print("Loss after epoch ", i, ": ", loss(y_pred, y, model_dict))

    return model_dict

In [19]:
model_dict = init_network()
model_dict = backpropagation(x, y, model_dict, epochs)

Loss after epoch  0 :  3.7731770446421424
Loss after epoch  50 :  2.6098052338814544
Loss after epoch  100 :  0.5219285469638569
Loss after epoch  150 :  0.1115980584273357
Loss after epoch  200 :  0.05187581036259295
Loss after epoch  250 :  0.03286383509181491
Loss after epoch  300 :  0.024012429974006017
Loss after epoch  350 :  0.019014473585694625
Loss after epoch  400 :  0.01584614723094639
Loss after epoch  450 :  0.013677157337235735
Loss after epoch  500 :  0.012108858230274038
Loss after epoch  550 :  0.01092765527416086
Loss after epoch  600 :  0.010009444812204947
Loss after epoch  650 :  0.009277481520202184
Loss after epoch  700 :  0.008681899934584328
Loss after epoch  750 :  0.008188970618961094
Loss after epoch  800 :  0.007775105484295284
Loss after epoch  850 :  0.007423338891578784
Loss after epoch  900 :  0.007121172269177832
Loss after epoch  950 :  0.006859205232347943


## Testing the network

In [20]:
df_test = pd.read_csv('Testing.csv')

In [21]:
x_test = df_test.iloc[:, 0:132].values
y_test = df_test.iloc[:, 132].values

In [22]:
x

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1]], dtype=int64)

In [23]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
y_test = one_hot_encode(y_test)
y_pred = predict(x_test, model_dict)

In [25]:
print("Loss on test set: ", loss(y_pred, y_test, model_dict))

Loss on test set:  0.3882679388057159


## Save the model

In [26]:
import pickle

filename = 'model.pkl'
pickle.dump(model_dict, open(filename, 'wb'))